In [1]:
cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [58]:
from preprocess import *

In [59]:
df

,sentences,labels,cleaned
0,Merrill Lynch analyst Campbell Morgan upgraded his recommendation on PaperlinX from `` neutral '' to `` buy '' in May .,2,merrill lynch analyst campbell morgan upgraded recommendation paperlinx from neutral to buy may
1,Eriikka S+Âderstr+Âm has previously held several positions in finance and control at Nokia Networks including acting as the Business Group Controller and having the corporate controller position at Nokia Siemens Networks .,1,eriikka s+âderstr+âm previously held several positions finance control nokia networks including acting business group controller corporate controller position nokia siemens networks
2,The webcast may be followed online on the company website at www.ruukki.com/investors .,1,webcast may followed online company website www.ruukki.com/investors
3,"Typical end-uses include roof structures , floorings , walls and ceilings , non-visible structures in vehicles , packaging and boxes , construction site structures , fencing and shelters , and formwork with a limited number of concrete pourings .",1,typical end-uses include roof structures floorings walls ceilings non-visible structures vehicles packaging boxes construction site structures fencing shelters formwork limited number concrete pourings
4,"The sale will be finalized in September or October , the company said .",1,sale finalized september october company said
...,...,...,...
1803,With this appointment Kaupthing Bank aims to further co-ordinate Capital Markets activities within the Group and to improve the overall service to clients .,2,appointment kaupthing bank aims to further co-ordinate capital markets activities within group to improve overall service to clients
1804,"Jon Risfelt has previously held operational executive positions in Europolitan , Ericsson , SAS , American Express card and travel divisions , as well as Nyman & Schultz ( CEO ) , Vodafone Sweden ( CEO ) , and Gambro Renal Products ( CEO ) .",1,jon risfelt previously held operational executive positions europolitan ericsson sas american express card travel divisions well nyman schultz ceo vodafone sweden ceo gambro renal products ceo
1805,"The group intends to relocate warehouse and office space in Loudeac and Saint Marcel to Morvillars , in the east of the country , where it already operates a hook manufacturing and distribution unit .",1,group intends to relocate warehouse office space loudeac saint marcel to morvillars east country already operates hook manufacturing distribution unit
1806,"The contract includes software licences , application maintenance and training .",1,contract includes software licences application maintenance training


In [60]:
df1 = df[['cleaned','labels']]
df1.rename(columns = {'cleaned':'sentences'}, inplace = True)


In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df1['sentences'],df1['labels'], test_size=0.2, random_state=420)

In [62]:
# train_json = train_data.to_json(orient='records')
# train_json

In [63]:
# pip install torchtext==0.9.0

In [64]:
# from torchtext.legacy import data
# from torchtext.legacy import datasets

# SENTENCE = data.Field()
# LABEL = data.Field()

In [65]:
# fields = {'sentences': ('s', SENTENCE), 'labels': ('l', LABEL)}

In [66]:
# train_data, valid_data= data.TabularDataset.splits(
#                                         path = '/content/drive/MyDrive/NLP_Project',
#                                         train = 'train.csv',
#                                         valid = 'valid.csv',
#                                         format = 'csv',
#                                         fields = fields,
#                                         skip_header = True)

In [67]:
train_data =list(zip(y_train,X_train))
valid_data =list(zip(y_test,X_test))

In [68]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [69]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = train_data
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [70]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

In [71]:
text_pipeline('this is an example')

[0, 0, 0, 1137]

In [72]:
label_pipeline('1')

1

In [116]:
vocab.__len__()

4801

In [73]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [80]:
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [81]:
train_iter1 = train_data
num_class = len(set([label for (label, text) in train_iter1]))
print(num_class)
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

3


In [82]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 10
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [83]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 50 # epoch
LR =0.1  # learning rate
BATCH_SIZE = 16 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_data
# test_iter2 =test_data 
valid_iter2= valid_data




train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
#                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |    10/   91 batches | accuracy    0.364
| epoch   1 |    20/   91 batches | accuracy    0.450
| epoch   1 |    30/   91 batches | accuracy    0.369
| epoch   1 |    40/   91 batches | accuracy    0.469
| epoch   1 |    50/   91 batches | accuracy    0.525
| epoch   1 |    60/   91 batches | accuracy    0.606
| epoch   1 |    70/   91 batches | accuracy    0.569
| epoch   1 |    80/   91 batches | accuracy    0.588
| epoch   1 |    90/   91 batches | accuracy    0.593
-----------------------------------------------------------
| end of epoch   1 | time:  0.36s | valid accuracy    0.591 
-----------------------------------------------------------
| epoch   2 |    10/   91 batches | accuracy    0.602
| epoch   2 |    20/   91 batches | accuracy    0.619
| epoch   2 |    30/   91 batches | accuracy    0.550
| epoch   2 |    40/   91 batches | accuracy    0.681
| epoch   2 |    50/   91 batches | accuracy    0.631
| epoch   2 |    60/   91 batches | accuracy    0.569
| epoch  

In [50]:
# print('Checking the results of test dataset.')
# accu_test = evaluate(test_dataloader)
# print('test accuracy {:8.3f}'.format(accu_test))

In [84]:
sentiment_label ={0: 'negative', 
                  1: 'neutral', 
                  2: 'positive'}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "He predicts that inflation will rise to an annual rate of 3 or 4 percent in coming years"
model = model.to("cpu")

print("This is a %s text" %sentiment_label[predict(ex_text_str, text_pipeline)])

This is a neutral text
